# Setup

In [1]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Precompiling project...


In [2]:
using RigidBodyDynamics

Please note that [RigidBodySim.jl](https://github.com/JuliaRobotics/RigidBodySim.jl) now provides a more capable simulation environment.

# Model definition

We'll just use the double pendulum model, loaded from a URDF:

In [3]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

# Controller

Let's write a simple controller that just applies $10 \sin(t)$ at the elbow joint and adds some damping at the shoulder joint:

In [4]:
shoulder, elbow = joints(mechanism)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, shoulder)] .= -1 .* velocity(state, shoulder)
    torques[velocity_range(state, elbow)] .= 10 * sin(t)
end

simple_control! (generic function with 1 method)

# Simulation

Basic simulation can be done using the `simulate` function. We'll first create a `MechanismState` object, and set the initial joint configurations and velocities:

In [5]:
state = MechanismState(mechanism)
zero_velocity!(state)
set_configuration!(state, shoulder, 0.7)
set_configuration!(state, elbow, -0.8);

Now we can simply call `simulate`, which will return a tuple consisting of:
* simulation times (a `Vector` of numbers)
* joint configuration vectors (a `Vector` of `Vector`s)
* joint velocity vectors (a `Vector` of `Vector`s)

In [6]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

For access to lower-level functionality, such as different ways of storing or visualizing the data generated during the simulation, it is advised to simply pattern match the basic `simulate` function.

# Visualization

For visualization, we'll use [`MeshCatMechanisms`](https://github.com/JuliaRobotics/MeshCatMechanisms.jl), an external package based on RigidBodyDynamics.jl.

(Note: the `#NBSKIP` comments are to skip these cells during `Pkg.test("RigidBodyDynamics")`)

In [ ]:
#NBSKIP
Pkg.activate(joinpath(@__DIR__, "visualization"));
pkg"instantiate"
pkg"precompile"
using MeshCatMechanisms

Create a `MechanismVisualizer` and open it in a new browser tab (see [`MeshCat.jl`](https://github.com/rdeits/MeshCat.jl) for other options):

In [ ]:
#NBSKIP
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))
open(mvis);

And animate:

In [ ]:
#NBSKIP
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);